In [73]:
import numpy as np
from keras.datasets import fashion_mnist
def get_fa_mnist(flatten=True):
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    if flatten:
        x_train = x_train.reshape(x_train.shape[0], 28*28)
        x_test = x_test.reshape(x_test.shape[0], 28*28)
    x_train = x_train.astype(np.float32) / 255.0
    x_test = x_test.astype(np.float32) / 255.0
    y_train = np.eye(10)[y_train]
    y_test = np.eye(10)[y_test]
    return (x_train, y_train), (x_test, y_test)

In [74]:
import numpy as np
class Activate:
    @staticmethod
    def leaky_relu(z, alpha=0.01):
        return np.where(z > 0, z, alpha*z)
    @staticmethod
    def leaky_relu_der(z, alpha=0.01):
        dz = np.ones_like(z)
        dz[z < 0] = alpha
        return dz
    @staticmethod
    def soft(z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

In [75]:
class FNN:
    def __init__(self, input_size, hidden_layers, output_size):
        np.random.seed(42)
        self.layers = [input_size] + hidden_layers + [output_size]
        self.weights = {}
        self.biases = {}
        for i in range(len(self.layers)-1):
            limit = np.sqrt(6 / (self.layers[i] + self.layers[i+1]))
            self.weights['W'+str(i+1)] = np.random.uniform(-limit, limit, (self.layers[i], self.layers[i+1]))
            self.biases['b'+str(i+1)] = np.zeros((1, self.layers[i+1]))
    def forw(self, X):
      self.cache = {'A0': X}
      L = len(self.layers)-1
      for l in range(1, L):
          Z = np.dot(self.cache['A'+str(l-1)], self.weights['W'+str(l)]) + self.biases['b'+str(l)]
          A = Activate.leaky_relu(Z)
          self.cache['Z'+str(l)] = Z
          self.cache['A'+str(l)] = A
      ZL = np.dot(self.cache['A'+str(L-1)], self.weights['W'+str(L)]) + self.biases['b'+str(L)]
      AL = Activate.soft(ZL)
      self.cache['Z'+str(L)] = ZL
      self.cache['A'+str(L)] = AL
      return AL
    def cross_ent(self,Y_pred, Y_true):
      m = Y_true.shape[0]
      return np.mean(-np.sum(Y_true * np.log(Y_pred + 1e-8), axis=1))
    def backw(self, Y_true, learning_rate=0.01):
      grads_w = {}
      grads_b = {}
      L = len(self.layers)-1
      m = Y_true.shape[0]
      dZ = self.cache['A'+str(L)] - Y_true
      grads_w['dW'+str(L)] = np.dot(self.cache['A'+str(L-1)].T, dZ)/m
      grads_b['db'+str(L)] = np.sum(dZ, axis=0, keepdims=True)/m
      dA_prev = np.dot(dZ, self.weights['W'+str(L)].T)
      for l in reversed(range(1, L)):
          dZ = dA_prev * Activate.leaky_relu_der(self.cache['Z'+str(l)])
          grads_w['dW'+str(l)] = np.dot(self.cache['A'+str(l-1)].T, dZ)/m
          grads_b['db'+str(l)] = np.sum(dZ, axis=0, keepdims=True)/m
          if l > 1:
              dA_prev = np.dot(dZ, self.weights['W'+str(l)].T)
      for l in range(1, L+1):
          self.weights['W'+str(l)] -= learning_rate * grads_w['dW'+str(l)]
          self.biases['b'+str(l)] -= learning_rate * grads_b['db'+str(l)]
    def train(self, X_train, Y_train, X_test, Y_test, lr=0.005, epochs=15):
      for epoch in range(epochs):
        Y_pred = self.forw(X_train)
        loss = self.cross_ent(Y_pred, Y_train)
        self.backw(Y_train, learning_rate=lr)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")
      Y_pred_test = self.forw(X_test)
      accuracy = np.mean(np.argmax(Y_pred_test, axis=1) == np.argmax(Y_test, axis=1))
      print(f"Test Accuracy: {accuracy*100:.2f}%")